# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 01/04/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Pre-requisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry]().

### Running on PSI


To Run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [1]:
!kubectl get ns | grep ska-mid-psi

ci-ska-mid-psi-1238980757-amjoshi                        Active   21h
ci-ska-mid-psi-1238980757-amjoshi-sdp                    Active   21h
ci-ska-mid-psi-1239108268-maecst                         Active   15h
ci-ska-mid-psi-1239108268-maecst-sdp                     Active   15h
ci-ska-mid-psi-1239127844-maecst                         Active   10m
ci-ska-mid-psi-1239127844-maecst-sdp                     Active   10m


To make future work a bit easier we can load this into a NS var.

In [2]:
# Change this var to be the non-spd namespace with your username in it
NS = "ci-ska-mid-psi-1239127844-maecst"

url = "https://142.73.34.170/"+NS+"/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print("You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run".format(NS))

You can monitor board status using: https://142.73.34.170/ci-ska-mid-psi-1239127844-maecst/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run


At this point, all coponents should be in the `disabled` state.

We can then check to ensure the pods have spun up successfully.

In [3]:
!kubectl -n $NS get pods

NAME                                            READY   STATUS             RESTARTS      AGE
databaseds-ds-databaseds-tango-base-0           1/1     Running            0             6m48s
databaseds-tangodb-databaseds-tango-base-0      1/1     Running            0             7m1s
ds-bite-bite-0                                  1/1     Running            0             6m23s
ds-cbfcontroller-controller-0                   1/1     Running            0             5m49s
ds-cbfsubarray-cbfsubarray-01-0                 1/1     Running            0             5m36s
ds-cbfsubarray-cbfsubarray-02-0                 1/1     Running            0             5m41s
ds-cbfsubarray-cbfsubarray-03-0                 1/1     Running            0             5m34s
ds-deployer-deployer-0                          1/1     Running            0             6m22s
ds-fsp-fsp-01-0                                 1/1     Running            0             6m1s
ds-fsp-fsp-02-0                                 1/1   

## Initial Setup/Using Deployer Device

First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [10]:
import os
# Set to the boards you wish to configure.
TARGET_BOARD = [1]

TANGO_HOST = "databaseds-tango-base."+NS+".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)

os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1239127844-maecst.svc.cluster.local:10000


We then load in the locations of local files for configuration. If you wish to use custom files you can change the folders pointed to, otherwise this code block can be run without edits.

In [11]:
# Get files from configs folder in repo
CURRDIR = os.path.dirname(os.getcwd())
CORRELATOR_FILE_LOCATION = os.path.abspath(os.path.join(CURRDIR,"configs/auto-correlation"))
COMMON_FILE_LOCATION = os.path.abspath(os.path.join(CURRDIR,"configs/common"))

Next, we pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code blocks can be used.

In [13]:
# Change this value if you want to specify a certain slim directory for files
slim_fs_config = ""
slim_vis_config = ""
if slim_fs_config != "" and slim_vis_config != "" :
    fs_config_path = os.path.join(CORRELATOR_FILE_LOCATION, slim_fs_config)
    vis_config_path = os.path.join(CORRELATOR_FILE_LOCATION, slim_vis_config)    
else:
    print("SLIM will use defaults for this test.")

SLIM will use defaults for this test.


In [ ]:
!"kubectl cp {fs_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml"
!"kubectl cp {vis_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml"

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [14]:
from PyTango import DeviceProxy, Database

# Setup the device proxies targeting bite and deployer
db = Database()
deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")

#Check the devices initially deployed to the database
devices = db.get_device_exported("*")
print(devices)
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

DbDatum[
        name = '*'
value_string = ['dserver/CbfController/controller', 'dserver/CbfSubarray/cbfsubarray-01', 'dserver/CbfSubarray/cbfsubarray-02', 'dserver/CbfSubarray/cbfsubarray-03', 'dserver/DataBaseds/2', 'dserver/ECBite/bite', 'dserver/ECDeployer/deployer', 'dserver/FspMulti/fsp-01', 'dserver/FspMulti/fsp-02', 'dserver/FspMulti/fsp-03', 'dserver/FspMulti/fsp-04', 'dserver/MidCspCapabilityFsp/capabilityfsp', 'dserver/MidCspCapabilityVcc/capabilityvcc', 'dserver/MidCspController/controller', 'dserver/MidCspSubarray/subarray1', 'dserver/MidCspSubarray/subarray2', 'dserver/MidCspSubarray/subarray3', 'dserver/PowerSwitch/powerswitch-001', 'dserver/PowerSwitch/powerswitch-002', 'dserver/PowerSwitch/powerswitch-003', 'dserver/PowerSwitch/powerswitch-004', 'dserver/SDPController/0', 'dserver/SDPQueueConnector/01', 'dserver/SDPSubarray/01', 'dserver/Slim/mesh', 'dserver/SlimLink/fs-links', 'dserver/SlimLink/vis-links', 'dserver/TalonBoard/talon-001', 'dserver/TalonBoard/talon-002'

Next we load in the hardware configuration depending on the talon boards selected. If a higher number board is chosen, we need to use the swapped config file.

In [15]:
if any(i > 4 for i in TARGET_BOARD):
    print("Using swap for higher number talons")
    config = "hw_config/hw_config_swap_psi.yaml"
else:
    print("Using standard HW config")
    config = "hw_config/hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(COMMON_FILE_LOCATION, config)

Using standard HW config


In [16]:
!kubectl cp $HW_CONFIG_FILE $NS/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

Currently using this step as a workaround for the deployer not generating a file that later steps use.

In [17]:
deployer_board_target = TARGET_BOARD[0]
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards=$deployer_board_target
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --download-artifacts
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --config-db

[talondx.py: line 487]INFO: User: root
[talondx.py: line 542]INFO: Generate talondx-config.json file
[talondx.py: line 487]INFO: User: root
[talondx.py: line 536]INFO: Download Artifacts
[talondx.py: line 330]INFO: Conan version: 1.62.0
[talondx.py: line 332]INFO: Conan local cache: There are no packages
[talondx.py: line 333]INFO: Clearing Conan local cache... WARN: No package recipe matches '*'
[talondx.py: line 336]INFO: Conan local cache: There are no packages
[talondx.py: line 339]INFO: DS Binary: ds-hps-master
[talondx.py: line 344]INFO: Conan info: {'package_name': 'dshpsmaster', 'user': 'nrc', 'channel': 'stable', 'version': '1.0.6', 'profile': 'conan_aarch64_profile.txt'}
Conan 1 is on a deprecation path, please consider migrating to Conan 2
Auto detecting your dev setup to initialize the default profile (/root/.conan/profiles/default)
No compiler was detected (one may not be needed)
Default settings
	os=Linux
	os_build=Linux
	arch=x86_64
	arch_build=x86_64
	build_type=Release

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [18]:
deployer_tango.targetTalons = TARGET_BOARD
print(deployer_tango.targetTalons)

[1]


With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [19]:
deployer_tango.generate_config_jsons()

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [20]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.download_artifacts()
except Exception as e:
    print(e)
    print("Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.")
deployer_tango.set_timeout_millis(4500)

DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_CallTimedout
  origin = Connection::command_inout()
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Timeout (200000 mS) exceeded on device mid_csp_cbf/ec/deployer, command download_artifacts
  origin = Connection::command_inout()
  reason = API_DeviceTimedOut
severity = ERR]
]
Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.


To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [21]:
!kubectl logs -n $NS ds-deployer-deployer-0 -f | grep 'Finished downloading'

1|2024-04-04T16:52:17.050Z|INFO|Dummy-1|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
^C


Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [22]:
deployer_tango.configure_db()

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [23]:
print(*db.get_device_exported("*").value_string, sep="\n")

dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray-01
dserver/CbfSubarray/cbfsubarray-02
dserver/CbfSubarray/cbfsubarray-03
dserver/DataBaseds/2
dserver/ECBite/bite
dserver/ECDeployer/deployer
dserver/FspMulti/fsp-01
dserver/FspMulti/fsp-02
dserver/FspMulti/fsp-03
dserver/FspMulti/fsp-04
dserver/MidCspCapabilityFsp/capabilityfsp
dserver/MidCspCapabilityVcc/capabilityvcc
dserver/MidCspController/controller
dserver/MidCspSubarray/subarray1
dserver/MidCspSubarray/subarray2
dserver/MidCspSubarray/subarray3
dserver/PowerSwitch/powerswitch-001
dserver/PowerSwitch/powerswitch-002
dserver/PowerSwitch/powerswitch-003
dserver/PowerSwitch/powerswitch-004
dserver/SDPController/0
dserver/SDPQueueConnector/01
dserver/SDPSubarray/01
dserver/Slim/mesh
dserver/SlimLink/fs-links
dserver/SlimLink/vis-links
dserver/TalonBoard/talon-001
dserver/TalonBoard/talon-002
dserver/TalonBoard/talon-003
dserver/TalonBoard/talon-004
dserver/TalonBoard/talon-005
dserver/TalonBoard/talon-006
dserver/Talon

## Setting Controller settings

Now that we have the controller device deployed, we can use it to prep for further steps.

In [24]:
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(CBF_tango.status())
control_tango = DeviceProxy("mid-csp/control/0")
print(control_tango.status())
subarray_tango = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
print(subarray_tango.status())
elt_controller_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(elt_controller_tango.status())

The device is in DISABLE state.
The device is in DISABLE state.
The device is in DISABLE state.
The device is in DISABLE state.


**Before running this step, ensure that you have checked out the boards you have selected to use!**

We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [25]:
# Set admin mode to online and sim mode to off on the cbf contoller
CBF_tango.adminMode = 0 
CBF_tango.simulationMode = 0
# Set admin mode to online and sim mode to off for subarray
subarray_tango.adminMode = 0 
subarray_tango.simulationMode = 0
# Set relevant values on the mid-csp controller
control_tango.adminMode = 0 
control_tango.simulationMode = 0
control_tango.cbfSimulationMode = 0

elt_controller_tango.adminMode = 0
elt_controller_tango.simulationMode = 0

if CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 0:
    print ("Set values successfully!")
elif CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 1:
    print ("Set to simulation mode on.")
else:
    print ("Error, couldn't set values!")


Set values successfully!


Checking the status dashboard, it should now display that all devices are both OFF and that the simulationstate is FALSE.

 Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command: 

In [26]:
init_sys_param_file = os.path.join(COMMON_FILE_LOCATION, "sys_params/initial_system_param.json")

with open(init_sys_param_file) as init_file:
    data = init_file.read()
# Use the InitSysParam command and feed in data
    
# add in check for off state before writing
upload_result = CBF_tango.InitSysParam(data)
print(upload_result[1])

['CbfController InitSysParam command completed OK']


We can now load in the dish configurations to the mid csp Controller, reading the json and passing it as a devstring to the appropriate command:

In [27]:
# Read in Json file from set configs folder
dish_config_file = os.path.join(COMMON_FILE_LOCATION,"sys_params/load_dish_config.json")
with open(dish_config_file) as file: 
    dish_config = file.read()
control_tango.LoadDishCfg(dish_config)

[array([2], dtype=int32), ['1712249750.2295868_102440398056115_LoadDishCfg']]

Now, we turn ON the Controller by passing it the device we want to turn on, and letting it run for 45ms to give the boards time to power on.

In [28]:
from time import sleep
print(control_tango.status())
control_tango.set_timeout_millis(50000)
target = ["mid_csp_cbf/sub_elt/controller"]
control_tango.On(target)
sleep(55)
print(control_tango.status())

The device is in OFF state.
The device is in ON state.


After running this step, check with the Tarnata dashboard to check that the boards are started.

## Running Commands Through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [29]:
target_mac_address = "08:c0:eb:9d:47:78"
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

RUNNING


Now that we have confirmed it is running, we load in what board we want to configure.

In [30]:
bite_tango.boards = TARGET_BOARD
bite_tango.bite_mac_address = target_mac_address

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [31]:
#print(bite_tango.command_inout("generate_bite_data"))

# For now run the backup
standin_board = TARGET_BOARD[0]
print(standin_board)
!kubectl exec -ti -n $NS ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$standin_board --bite_mac_address=$target_mac_address

1
[midcbf_bite.py: line 43]INFO: User: root
[midcbf_bite.py: line 118]INFO: Test parameter files have not all been copied to, or are not all accessible at, their expected location. Cannot proceed with specified test parameters unless all files are present, since some files would be pointing to non-existent information expected in other files. Resorting to the default test parameters in basic_test_parameters.json.
[midcbf_bite.py: line 172]INFO: No test parameter arguments provided. Defaulting to basic test parameters on boards: ['1']
[midcbf_bite.py: line 182]INFO: BITE receptors: [{'talon': '1', 'dish_id': 'SKA001', 'sample_rate_k': 100, 'bite_config_id': 'basic gaussian noise', 'bite_initial_timestamp_time_offset': 60.0}]
[midcbf_bite.py: line 203]INFO: Talon BITE Configure
[midcbf_bite.py: line 118]INFO: Creating dps for device server: dsgaussiannoisegen
[midcbf_bite.py: line 680]INFO: Creating device server for talondx-001/gaussiannoisegen/gn_gen_src_polX_0
[midcbf_bite.py: line 66

In [ ]:
!kubectl -n $NS logs ds-bite-bite-0 | grep generate_bite_data

With the bite data generated, we can then start generating LSTV playback data:

In [32]:
bite_tango.start_lstv_replay()

'Started LSTV replay.'

We can then check the logs to ensure the LSTV geneartion runs correctly.

In [33]:
!kubectl -n $NS logs ds-bite-bite-0 | grep start_lstv_replay

1|2024-04-04T17:03:13.970Z|INFO|Dummy-1|start_lstv_replay|bite.py#381||Starting LSTV replay...
1|2024-04-04T17:03:13.970Z|INFO|Dummy-1|start_lstv_replay|bite.py#401||Started LSTV replay.


## Assigning Resources

Next, we use the relevant subarray device to assign resources. First, as with the other devices, we establish a DeviceProxy to connect to it. We also read in the assign_resources file to load in using the command. 

In [34]:
subarray_tango = DeviceProxy("mid-csp/subarray/01")
print(subarray_tango.State())
assign_resources_file = os.path.join(CORRELATOR_FILE_LOCATION, "assign_resources.json")
with open(assign_resources_file) as init_file:
    resource_data = init_file.read()
subarray_tango.On()
print(resource_data)
print(subarray_tango.State())

ON
{
	"interface": "https://schema.skao.int/ska-csp-assignresources/2.2",
	"subarray_id": 1,
	"dish":{
		"receptor_ids":["SKA001"]
	}
}

ON


Next,we run the actual command to pass in the resources:

In [35]:
subarray_tango.AssignResources(resource_data)

[array([2], dtype=int32),
 ['1712250209.360067_115571144741173_AssignResources']]

## SDP Setup

Next, we run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

First, we get the files needed to set up the SDP, if you require custom files, load in new ones or change the commands to point to them.

In [36]:
# Change this if you want to use a different folder for files
SPD_FOLDER = CORRELATOR_FILE_LOCATION

sdp_assign_resources_file = os.path.join(SPD_FOLDER,"assign_resources_sdp.json")
sdp_configure_file = os.path.join(SPD_FOLDER,"configure_sdp.json")
sdp_scan_file= os.path.join(SPD_FOLDER,"scan_sdp.json")
if not os.path.isfile(sdp_scan_file) or not os.path.isfile(sdp_configure_file) or not os.path.isfile(sdp_assign_resources_file):
    print("Warning, one or more of the files pointed to doesn't exist!")
else: 
    print("All files loaded correctly")

All files loaded correctly


Next, we set up a proxy to the SDP device:

In [37]:
sdp_tango = DeviceProxy("test-sdp/subarray/01")
print(sdp_tango.Status())
sdp_tango.On()

The device is in OFF state.


Now, we assign resources as we did with the subarray:

In [38]:
with open(sdp_assign_resources_file) as f:
    sdp_resources = f.read()
sdp_tango.AssignResources(sdp_resources)

Next we send the configure command along with the required config file:

In [42]:
with open(sdp_configure_file) as f:
    sdp_configuration = f.read()
sdp_tango.Configure(sdp_configuration)

After running this step, a pod for reciving visabilities will be launched, grab the IP of this pod and place it in the output_host var in the configure_scan json file.

In [43]:
# Grab the pod name
!kubectl -n $NS-sdp get pods | grep vis-receive

proc-pb-test-20211111-00019-vis-receive-00-0   0/3     ContainerCreating   0          21s


In [67]:
# Then find the IP:
vis_pod = "proc-pb-test-20211111-00019-vis-receive-00-0"
!kubectl -n $NS-sdp describe pod $vis_pod
!kubectl -n $NS-sdp describe pod $vis_pod | grep "ips":

Name:         proc-pb-test-20211111-00019-vis-receive-00-0
Namespace:    ci-ska-mid-psi-1239127844-maecst-sdp
Priority:     0
Node:         rmdskadevdu013/142.73.34.172
Start Time:   Thu, 04 Apr 2024 17:06:26 +0000
Labels:       app.kubernetes.io/instance=proc-pb-test-20211111-00019-vis-receive
              app.kubernetes.io/name=vis-receive
              apps.kubernetes.io/pod-index=0
              controller-revision-hash=proc-pb-test-20211111-00019-vis-receive-00-556966dc6f
              script=vis-receive
              statefulset.kubernetes.io/pod-name=proc-pb-test-20211111-00019-vis-receive-00-0
Annotations:  cni.projectcalico.org/containerID: bb85e4d87c03e0ef14b0df7a8890867377972cb2982e0f39869e60a4ece55d92
              cni.projectcalico.org/podIP: 10.10.47.246/32
              cni.projectcalico.org/podIPs: 10.10.47.246/32
              k8s.v1.cni.cncf.io/network-status:
                [{
                    "name": "k8s-pod-network",
                    "ips": [
             

With this pod, we will also want to prep for mointoring the pod for when it (hopefully) recivies the visibilties

Run this command, then use the output in a seperate terminal to enter the correct pod:

In [49]:
!echo kubectl exec -n $NS-sdp -ti $vis_pod -- bash


kubectl exec -n ci-ska-mid-psi-1239127844-maecst-sdp -ti proc-pb-test-20211111-00019-vis-receive-00-0 -- bash


We can then run the following commands in this pod to start monitoring for the correct traffic:

If needed, grab the visibility IP from the sdp namespace's test script pod, and input that into the IP set in the scan configuration

Now on the CBF side we can load in the corresponding Scan file.

In [50]:
cbf_configure_scan_file = os.path.join(CORRELATOR_FILE_LOCATION,"configure_scan.json")
cbf_scan_file  = os.path.join(CORRELATOR_FILE_LOCATION,"scan.json")
with open(cbf_configure_scan_file) as f:
    cbf_config = f.read()
with open(cbf_scan_file) as f:
    cbf_scan = f.read()
print("Observation state: {}".format(subarray_tango.obsState))
subarray_tango.Configure(cbf_config)
print("Observation state: {}".format(subarray_tango.obsState))

Observation state: 2
Observation state: 3


As a temporary step, load this into the delay model on the CSP Subarray dash:

And finally we start the SDP output via the Scan command:

In [62]:
with open(sdp_scan_file) as f:
    sdp_scan = f.read()
sdp_tango.Scan(sdp_scan)

And then the matching command in the subarray:

In [63]:
subarray_tango.Scan(cbf_scan)

[array([2], dtype=int32), ['1712252128.5460978_117267679238535_Scan']]

In [65]:
print("Observation state: {}".format(subarray_tango.obsState))

Observation state: 4


Checking the dashboard, the dish should now be in the SCANNING status.

## Checking Visabilties


## Cleanup

In [64]:
sdp_tango.EndScan()
subarray_tango.EndScan()

[array([2], dtype=int32), ['1712255575.6340024_156133748132558_EndScan']]

In [ ]:
subarray_tango.Off()
CBF_tango.Off()
control_tango.Off()

Now that we're done, free up dev resources on PSI by deleting your ns.

In [ ]:
!kubectl delete ns $NS
!kubectl delete ns $NS-sdp

In [ ]:
!kubectl get ns | grep ska-mid-psi

If the boards are on, use the LRU command to turn them off.